<a href="https://colab.research.google.com/github/SelenaNahra/DL/blob/main/HW3P2LSTM20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np
import requests
import time

# Step 1: Download the dataset
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)
text = response.text  # This is the entire text data

# Step 2: Prepare the dataset
sequence_length = 20
# Create a character mapping to integers
chars = sorted(list(set(text)))
char_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_char = {i: ch for i, ch in enumerate(chars)}

# Encode the text into integers
encoded_text = [char_to_int[ch] for ch in text]

# Create sequences and targets
sequences = []
targets = []
for i in range(0, len(encoded_text) - sequence_length):
    seq = encoded_text[i:i+sequence_length]
    target = encoded_text[i+sequence_length]
    sequences.append(seq)
    targets.append(target)

# Convert lists to PyTorch tensors
sequences = torch.tensor(sequences, dtype=torch.long)
targets = torch.tensor(targets, dtype=torch.long)

# Step 3: Create a dataset class
class CharDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = sequences
        self.targets = targets

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, index):
        return self.sequences[index], self.targets[index]

# Instantiate the dataset
dataset = CharDataset(sequences, targets)

# Step 4: Create data loaders
batch_size = 128
train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

# Defining the RNN model
class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(CharRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.rnn(embedded)
        output = self.fc(output[:, -1, :])
        return output

# Hyperparameters
hidden_size = 128
learning_rate = 0.005
epochs = 100

# Model, loss, and optimizer
model = CharRNN(len(chars), hidden_size, len(chars)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training the model
start_time = time.time()

for epoch in range(epochs):
    model.train()
    for sequences, targets in train_loader:  # Iterate over batches
        sequences, targets = sequences.to(device), targets.to(device)  # Move data to device
        optimizer.zero_grad()
        output = model(sequences)
        loss = criterion(output, targets)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_loss = 0
        correct = 0
        total = 0
        for sequences, targets in test_loader:
            sequences, targets = sequences.to(device), targets.to(device)  # Move data to device
            val_output = model(sequences)
            val_loss += criterion(val_output, targets).item()
            _, predicted = torch.max(val_output.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
        val_loss /= len(test_loader)
        val_accuracy = correct / total

    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item()}, Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

end_time = time.time()
training_time = end_time - start_time

# Prediction function
def predict_next_char(model, char_to_ix, ix_to_char, initial_str):
    model.eval()
    with torch.no_grad():
        initial_input = torch.tensor([char_to_ix[c] for c in initial_str[-sequence_length:]], dtype=torch.long).unsqueeze(0).to(device)
        prediction = model(initial_input)
        predicted_index = torch.argmax(prediction, dim=1).item()
        return ix_to_char[predicted_index]

# Counting model complexity
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Predicting the next character
test_str = "This is a simple example to demonstrate how to predict the next char"
predicted_char = predict_next_char(model, char_to_int, int_to_char, test_str)
print(f"Predicted next character: '{predicted_char}'")
print(f"Training time: {training_time} seconds")
print(f"Number of trainable parameters in the model: {count_parameters(model)}")

# Compute computational complexity
embedding_complexity = sequences.shape[1] * hidden_size
rnn_complexity = sequences.shape[1] * hidden_size * hidden_size
linear_complexity = hidden_size * len(chars)

total_complexity = embedding_complexity + rnn_complexity + linear_complexity
print(f"Total computational complexity: {total_complexity}")

Epoch 10, Loss: 1.7575589418411255, Validation Loss: 1.5623016992794394, Validation Accuracy: 0.5285486943852964
Epoch 20, Loss: 1.4294040203094482, Validation Loss: 1.6159882358073914, Validation Accuracy: 0.5143651238372745
Epoch 30, Loss: 1.5688529014587402, Validation Loss: 1.6624127107314932, Validation Accuracy: 0.49907430236467554
Epoch 40, Loss: 2.040755271911621, Validation Loss: 1.691041222332677, Validation Accuracy: 0.4954118569987672
Epoch 50, Loss: 1.7211724519729614, Validation Loss: 1.7046486730898378, Validation Accuracy: 0.4906914714782024
Epoch 60, Loss: 1.0184515714645386, Validation Loss: 1.7049642233621507, Validation Accuracy: 0.4923994172363555
Epoch 70, Loss: 1.995439052581787, Validation Loss: 1.7099102693115646, Validation Accuracy: 0.4859262579849826
Epoch 80, Loss: 1.4290162324905396, Validation Loss: 1.7195627034155034, Validation Accuracy: 0.4871455788411969
Epoch 90, Loss: 1.9586894512176514, Validation Loss: 1.7186001736886598, Validation Accuracy: 0.48

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np
import requests
import time

# Step 1: Download the dataset
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)
text = response.text  # This is the entire text data

# Step 2: Prepare the dataset
sequence_length = 30
# Create a character mapping to integers
chars = sorted(list(set(text)))
char_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_char = {i: ch for i, ch in enumerate(chars)}

# Encode the text into integers
encoded_text = [char_to_int[ch] for ch in text]

# Create sequences and targets
sequences = []
targets = []
for i in range(0, len(encoded_text) - sequence_length):
    seq = encoded_text[i:i+sequence_length]
    target = encoded_text[i+sequence_length]
    sequences.append(seq)
    targets.append(target)

# Convert lists to PyTorch tensors
sequences = torch.tensor(sequences, dtype=torch.long)
targets = torch.tensor(targets, dtype=torch.long)

# Step 3: Create a dataset class
class CharDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = sequences
        self.targets = targets

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, index):
        return self.sequences[index], self.targets[index]

# Instantiate the dataset
dataset = CharDataset(sequences, targets)

# Step 4: Create data loaders
batch_size = 128
train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

# Defining the RNN model
class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, fc_hidden_size):
        super(CharRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.LSTM(hidden_size, hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, fc_hidden_size)
        self.fc2 = nn.Linear(fc_hidden_size, output_size)

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.rnn(embedded)
        output = self.fc(output[:, -1, :])
        output = self.fc2(output)
        return output

# Hyperparameters with different values
hidden_size = 256  # Hidden size of 512
num_layers = 4  # Four hidden layers
fc_hidden_size = 128  # FC hidden size of 128

learning_rate = 0.005
epochs = 100

# Model, loss, and optimizer
model = CharRNN(len(chars), hidden_size, len(chars), num_layers, fc_hidden_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training the model
start_time = time.time()

for epoch in range(epochs):
    model.train()
    for sequences, targets in train_loader:  # Iterate over batches
        sequences, targets = sequences.to(device), targets.to(device)  # Move data to device
        optimizer.zero_grad()
        output = model(sequences)
        loss = criterion(output, targets)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_loss = 0
        correct = 0
        total = 0
        for sequences, targets in test_loader:
            sequences, targets = sequences.to(device), targets.to(device)  # Move data to device
            val_output = model(sequences)
            val_loss += criterion(val_output, targets).item()
            _, predicted = torch.max(val_output.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
        val_loss /= len(test_loader)
        val_accuracy = correct / total

    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item()}, Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

end_time = time.time()
training_time = end_time - start_time

# Prediction function
def predict_next_char(model, char_to_ix, ix_to_char, initial_str):
    model.eval()
    with torch.no_grad():
        initial_input = torch.tensor([char_to_ix[c] for c in initial_str[-sequence_length:]], dtype=torch.long).unsqueeze(0).to(device)
        prediction = model(initial_input)
        predicted_index = torch.argmax(prediction, dim=1).item()
        return ix_to_char[predicted_index]

# Predicting the next character
test_str = "This is a simple example to demonstrate how to predict the next char"
predicted_char = predict_next_char(model, char_to_int, int_to_char, test_str)
print(f"Predicted next character: '{predicted_char}'")
print(f"Training time: {training_time} seconds")
print(f"Number of trainable parameters in the model: {count_parameters(model)}")

# Compute computational complexity
embedding_complexity = sequences.shape[1] * hidden_size
rnn_complexity = sequences.shape[1] * hidden_size * hidden_size * num_layers
linear_complexity = fc_hidden_size * hidden_size + fc_hidden_size * len(chars)

total_complexity = embedding_complexity + rnn_complexity + linear_complexity
print(f"Total computational complexity: {total_complexity}")

Epoch 10, Loss: 3.3239777088165283, Validation Loss: 1.3597345812116897, Validation Accuracy: 0.5855527114442357
Epoch 20, Loss: 0.14436347782611847, Validation Loss: 1.3490731900133623, Validation Accuracy: 0.5908334939683422
Epoch 30, Loss: 0.8805272579193115, Validation Loss: 1.3568773512107095, Validation Accuracy: 0.5910038417917005
Epoch 40, Loss: 1.9022396802902222, Validation Loss: 1.3630190377117777, Validation Accuracy: 0.5888251827876974
Epoch 50, Loss: 0.9881043434143066, Validation Loss: 1.3641987340380526, Validation Accuracy: 0.5880855146073258
Epoch 60, Loss: 0.6330987811088562, Validation Loss: 1.37209538844421, Validation Accuracy: 0.5875610226248806
Epoch 70, Loss: 1.8831299543380737, Validation Loss: 1.3724477799268406, Validation Accuracy: 0.5891972582966114
Epoch 80, Loss: 0.7058465480804443, Validation Loss: 1.3810669583831103, Validation Accuracy: 0.5890493246605372
Epoch 90, Loss: 0.033092200756073, Validation Loss: 1.3814509812265856, Validation Accuracy: 0.58

NameError: name 'count_parameters' is not defined